 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/rnn_test.ipynb#Set-up-data" data-toc-modified-id="Set-up-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set up data</a></span></li><li><span><a href="http://localhost:8888/notebooks/rnn_test.ipynb#Build-model" data-toc-modified-id="Build-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Build model</a></span></li></ul></div>

In [102]:
# Imports
from importlib import reload
import crps_loss; reload(crps_loss)
from crps_loss import crps_cost_function, crps_cost_function_seq
import utils; reload(utils)
from utils import *
import matplotlib.pyplot as plt
%matplotlib inline

import keras
from keras.layers import Input, Dense, merge, Embedding, Flatten, Dropout, \
    SimpleRNN, LSTM, TimeDistributed, GRU
from keras.layers.merge import Concatenate
from keras.models import Model, Sequential
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, Adam

In [6]:
keras.backend.backend()

'tensorflow'

In [7]:
# Basic setup
DATA_DIR = '/Volumes/STICK/data/ppnn_data/'  # Mac
# DATA_DIR = '/project/meteo/w2w/C7/ppnn_data/'   # LMU
results_dir = '../results/'
window_size = 25   # Days in rolling window
fclt = 48   # Forecast lead time in hours

## Set up data

In [8]:

train_dates = ['2015-01-01', '2016-01-01']
test_dates =  ['2016-01-01', '2017-01-01']
train_set, test_set = get_train_test_sets(DATA_DIR, train_dates, test_dates)

train set contains 365 days
test set contains 366 days


In [9]:
train_set.features.shape

(180849, 2)

In [10]:
targets, features, dates, station_id, feature_names = load_raw_data(DATA_DIR)

In [11]:
targets.shape, features.shape

((3653, 537), (2, 3653, 537))

In [12]:
train_dates_idxs = [return_date_idx(dates, train_dates[0]),
                            return_date_idx(dates, train_dates[1])]
test_dates_idxs = [return_date_idx(dates, test_dates[0]),
                           return_date_idx(dates, test_dates[1])]

In [13]:
train_dates_idxs

[2920, 3285]

In [60]:
x_train = features[:, train_dates_idxs[0]:train_dates_idxs[1]]
y_train = targets[train_dates_idxs[0]:train_dates_idxs[1]]

In [61]:
x_train.shape, y_train.shape

((2, 365, 537), (365, 537))

In [62]:
x_test = features[:, test_dates_idxs[0]:test_dates_idxs[1]]
y_test = targets[test_dates_idxs[0]:test_dates_idxs[1]]

In [63]:
x_test.shape, y_test.shape

((2, 366, 537), (366, 537))

In [67]:
def prep_seq_data(x, y, seq_len):
    # Stack sequences [seq, feature, day, station]
    x_seq = np.stack([x[:, i:-(seq_len-i)] for i in range(seq_len)])
    y_seq = np.stack([y[i:-(seq_len-i)] for i in range(1, seq_len)] + [y[seq_len:]])
    
    # Ravel arrays [seq, feature, instance]
    x_seq = np.reshape(x_seq, (seq_len, 2, -1))
    y_seq = np.reshape(y_seq, (seq_len, -1))
    
    # Roll arrays[sample, time step, feature]
    x_seq = np.rollaxis(x_seq, 2, 0)
    y_seq = np.rollaxis(y_seq, 1, 0)
    y_seq = np.atleast_3d(y_seq)
    
    # Remove nans
    mask = np.isfinite(np.sum(y_seq, axis=(1,2)))
    x_seq = x_seq[mask]
    y_seq = y_seq[mask]
    
    return x_seq, y_seq

In [68]:
# Reshape them
seq_len = 5

In [69]:
x_seq_train, y_seq_train = prep_seq_data(x_train, y_train, seq_len)

In [70]:
x_seq_test, y_seq_test = prep_seq_data(x_test, y_test, seq_len)

In [71]:
x_seq_train.shape, y_seq_train.shape

((177473, 5, 2), (177473, 5, 1))

## Build model

In [81]:
batch_size = 64
hidden_nodes = 100

In [82]:
inp = Input(shape=(seq_len, 2, )) # time step, feature
x = LSTM(hidden_nodes, return_sequences=True)(inp)
x = TimeDistributed(Dense(2, activation='linear'))(x)
rnn_model = Model(inputs=inp, outputs=x)

In [83]:
rnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 5, 2)              0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 5, 100)            41200     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 5, 2)              202       
Total params: 41,402
Trainable params: 41,402
Non-trainable params: 0
_________________________________________________________________


In [84]:
rnn_model.compile(optimizer=Adam(0.001), loss=crps_cost_function_seq)

In [87]:
rnn_model.fit(x_seq_train, y_seq_train, epochs=1, batch_size=1024,
              validation_data=(x_seq_test, y_seq_test))

Train on 177473 samples, validate on 178978 samples
Epoch 1/1
177473/177473 [==============================] - 51s - loss: 1.7016 - val_loss: 1.6561


In [103]:
inp = Input(shape=(seq_len, 2, )) # time step, feature
x = GRU(hidden_nodes)(inp)
x = Dense(2, activation='linear')(x)
rnn_model2 = Model(inputs=inp, outputs=x)

In [104]:
rnn_model2.compile(optimizer=Adam(0.001), loss=crps_cost_function)

In [105]:
rnn_model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 5, 2)              0         
_________________________________________________________________
gru_1 (GRU)                  (None, 100)               30900     
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 202       
Total params: 31,102
Trainable params: 31,102
Non-trainable params: 0
_________________________________________________________________


In [ ]:
rnn_model2.fit(x_seq_train, y_seq_train[:,-1], epochs=5, batch_size=1024,
              validation_data=(x_seq_test, y_seq_test[:,-1]))

Train on 177473 samples, validate on 178978 samples
Epoch 1/1
177152/177473 [============================>.] - ETA: 0s - loss: 2.6584